# Beta, Sharpe Ratio, and Excess Returns (Ann)

$R_i - R_f = \alpha_i + \beta_i * (R_M - R_f) + \epsilon_{i}$

- $\beta_i > 1$:   asset moves in the same direction as the benchmark but is more volatile.
- $\beta_i = 1$:   asset moves identical as the benchmark.
- $0 < \beta_i < 1$:   asset moves in the same direction as the benchmark but is less volatile.
- $\beta_i < 0$:   asset moves in the opposite direction as the benchmark.
    - |$\beta_i$| > 1: asset is more volatile.
    - |$\beta_i$| < 1: asset is less volatile.
    
Sharpe Ratio = $\frac{R_i - R_f}{\sigma_i}$, where $\sigma_i$ is the standard deviation of the asset's excess return.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from duneanalytics import DuneAnalytics
import numpy as np
import pandas as pd
import pandas_datareader.data as reader
import datetime as dt
import statsmodels.api as sm
import dataframe_image as dfi # for saving styled data frame print-out table as png

2022-08-08 20:33:21,117 : INFO : _init_num_threads : NumExpr defaulting to 8 threads.


In [2]:
# set paths and create dirs 
base_dir = '../..'
output_dir = os.path.join(base_dir, 'output')
png_dir = os.path.join(output_dir, 'png')
os.makedirs(png_dir, exist_ok=True)

### Define Helper Functions

In [3]:
def extract_frame_from_dune_data(dune_data, date_col='day'):    
    dd = dune_data['data']['get_result_by_result_id']
    df = pd.json_normalize(dd, record_prefix='')
    df = df.loc[:, df.columns.str.startswith('data')]
    df.columns = df.columns.str.replace('data.', '', regex=False)
    df['date'] = pd.to_datetime(df[date_col].str.replace('T.*', '', regex=True))
    if date_col != 'date':
        df = df.drop(date_col, axis=1)
    df = df.set_index('date')
    # drop the last row cuz it may not always be a full day
    return df.iloc[:-1, :]

In [4]:
def calc_beta(df_ret, token='BTC', benchmark='SP500'):
    da = df_ret[[benchmark, token]].dropna()
    X = da[benchmark]
    y = da[token]
    X_sm = sm.add_constant(X)
    model = sm.OLS(y, X_sm)
    results = model.fit()
    return results.params[benchmark]

In [5]:
def annualize_tot_ret(tot_ret, dur_years):
    return (1+tot_ret)**(1/dur_years) - 1

## Get GLP and TriCrypto Prices from Dune

In [6]:
# get Dune Analytics login credentials
MY_USERNAME = os.environ.get('DUNE_USERNAME')
MY_PASSWORD = os.environ.get('DUNE_PASSWORD')
dune = DuneAnalytics(MY_USERNAME, MY_PASSWORD)

dune.login()
dune.fetch_auth_token()
 
# fetch query result
glp_arbi_prices = dune.query_result(dune.query_result_id(query_id=1069389))
tricrypto_prices = dune.query_result(dune.query_result_id(query_id=1145739))

In [7]:
# get GLP price data from Arbitrum 
df_glp_arbi_prices = extract_frame_from_dune_data(glp_arbi_prices, 'date').rename({'price':'GLP'}, axis=1)
print(df_glp_arbi_prices.head(), '\n\n')
print(df_glp_arbi_prices.tail())

                 GLP
date                
2021-08-31  1.178252
2021-09-01  1.259457
2021-09-02  1.278878
2021-09-03  1.315815
2021-09-04  1.308367 


                 GLP
date                
2022-08-03  0.945496
2022-08-04  0.936071
2022-08-05  0.948903
2022-08-06  0.957873
2022-08-07  0.953667


In [8]:
# get TriCrypto price data from Ethereum
df_tricrypto_prices = extract_frame_from_dune_data(tricrypto_prices, 'date').rename({'price':'TriCrypto'}, axis=1)
print(df_tricrypto_prices.head(), '\n\n')
print(df_tricrypto_prices.tail())

              TriCrypto
date                   
2021-06-09  1370.735971
2021-06-10  1371.336326
2021-06-11  1345.940569
2021-06-12  1321.675295
2021-06-13  1321.711089 


              TriCrypto
date                   
2022-08-03  1033.324403
2022-08-04  1022.775117
2022-08-05  1036.296811
2022-08-06  1045.616781
2022-08-07  1038.269638


## Get Price Data from Yahoo

SP500, Tips, Bond, Gold, Reit, BTC, and ETH 

In [9]:
years = 2 # how many years of data do you want to get?

In [10]:
# get prices from yahoo
end = dt.datetime.now()
start = dt.date(end.year - years, end.month, end.day) 
df_prices = (reader.get_data_yahoo(['^GSPC', 'TIP', 'BND', 'VNQ', 'GLD', 'BTC-USD', 'ETH-USD'], start, end)['Adj Close']
                 .rename({'^GSPC':'SP500', 'TIP':'Inflation-Linked Bonds', 'BND':'Nominal Bonds', 
                          'VNQ':'Real Estate', 'GLD':'Gold', 'BTC-USD':'BTC', 'ETH-USD':'ETH'}, axis=1))
df_prices.columns.name = None
print(df_prices.head(), '\n\n')
print(df_prices.tail())

                  SP500  Inflation-Linked Bonds  Nominal Bonds  Real Estate  \
Date                                                                          
2020-08-07  3351.280029              114.615196      85.484802    77.286713   
2020-08-08          NaN                     NaN            NaN          NaN   
2020-08-09          NaN                     NaN            NaN          NaN   
2020-08-10  3360.469971              114.478935      85.389160    77.408653   
2020-08-11  3333.689941              114.224586      85.092682    76.292374   

                  Gold           BTC         ETH  
Date                                              
2020-08-07  190.809998  11601.472656  379.512848  
2020-08-08         NaN  11754.045898  393.987366  
2020-08-09         NaN  11675.739258  391.120453  
2020-08-10  190.149994  11878.111328  395.887573  
2020-08-11  179.940002  11410.525391  380.384064   


                  SP500  Inflation-Linked Bonds  Nominal Bonds  Real Estate  \
Date   

In [11]:
# get risk free rate
rfs = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0].RF
# convert Period index to datetime
rfs.index = pd.to_datetime(rfs.index.to_timestamp(how='end').strftime('%Y-%m-%d'))
print(rfs.head(), '\n\n')
print(rfs.tail())

Date
2020-08-31    0.01
2020-09-30    0.01
2020-10-31    0.01
2020-11-30    0.01
2020-12-31    0.01
Name: RF, dtype: float64 


Date
2022-02-28    0.00
2022-03-31    0.01
2022-04-30    0.01
2022-05-31    0.03
2022-06-30    0.06
Name: RF, dtype: float64


## Calculate Daily, Weekly, Monthly Returns

In [12]:
daily_rets = dict()
monthly_rets = dict()
weekly_rets = dict()
for col in df_prices.columns:
    print(col)
    daily_rets_ticker = df_prices[col].pct_change().dropna()
    monthly_rets_ticker = daily_rets_ticker.resample('M').agg(lambda x: (1+x).prod()-1).iloc[1:-1] # drop 1st and last row since they may not be a full month
    print("Month Range:", monthly_rets_ticker.index.min().strftime('%Y-%m-%d'), '~', 
          monthly_rets_ticker.index.max().strftime('%Y-%m-%d'))
    weekly_rets_ticker = daily_rets_ticker.resample('W').agg(lambda x: (1+x).prod()-1).iloc[1:-1] # drop 1st and last row since they may not be a full week
    print("Week Range:", weekly_rets_ticker.index.min().strftime('%Y-%W'), '~', 
          weekly_rets_ticker.index.max().strftime('%Y-%W'))
    # collect results
    daily_rets[col] = daily_rets_ticker
    monthly_rets[col] = monthly_rets_ticker
    weekly_rets[col] = weekly_rets_ticker
    
daily_rets = pd.DataFrame(daily_rets)    
monthly_rets = pd.DataFrame(monthly_rets)
weekly_rets = pd.DataFrame(weekly_rets)

SP500
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31
Inflation-Linked Bonds
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31
Nominal Bonds
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31
Real Estate
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31
Gold
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31
BTC
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31
ETH
Month Range: 2020-09-30 ~ 2022-07-31
Week Range: 2020-32 ~ 2022-31


In [13]:
# GLP daily and monthly returns
daily_rets_glp = df_glp_arbi_prices.GLP.pct_change().dropna()
monthly_rets_glp = daily_rets_glp.resample('M').agg(lambda x: (1+x).prod()-1).iloc[:-1] # drop last row since it may not be a full month, do not drop 1st row since it's a full month for GLP
weekly_rets_glp = daily_rets_glp.resample('W').agg(lambda x: (1+x).prod()-1).iloc[1:-1] # drop first and last rows since they may not be a full week
print("Month Range:", monthly_rets_glp.index.min().strftime('%Y-%m-%d'), '~', 
      monthly_rets_glp.index.max().strftime('%Y-%m-%d'))
print("Week Range:", weekly_rets_glp.index.min().strftime('%Y-%W'), '~', 
      weekly_rets_glp.index.max().strftime('%Y-%W'))

Month Range: 2021-09-30 ~ 2022-07-31
Week Range: 2021-36 ~ 2022-30


In [14]:
# TriCrypto daily and monthly returns
daily_rets_tri = df_tricrypto_prices.TriCrypto.pct_change().dropna()
monthly_rets_tri = daily_rets_tri.resample('M').agg(lambda x: (1+x).prod()-1).iloc[1:-1] # drop 1st and last row since they may not be a full month
weekly_rets_tri = daily_rets_tri.resample('W').agg(lambda x: (1+x).prod()-1).iloc[1:-1] # drop first and last rows since they may not be a full week
print("Month Range:", monthly_rets_tri.index.min().strftime('%Y-%m-%d'), '~', 
      monthly_rets_tri.index.max().strftime('%Y-%m-%d'))
print("Week Range:", weekly_rets_tri.index.min().strftime('%Y-%W'), '~', 
      weekly_rets_tri.index.max().strftime('%Y-%W'))

Month Range: 2021-07-31 ~ 2022-07-31
Week Range: 2021-24 ~ 2022-30


In [15]:
# join all returns
daily_rets = daily_rets.join(daily_rets_glp).join(daily_rets_tri)
monthly_rets = monthly_rets.join(monthly_rets_glp).join(monthly_rets_tri)
weekly_rets = weekly_rets.join(weekly_rets_glp).join(weekly_rets_tri)

In [16]:
print(daily_rets.head(), '\n\n')
print(daily_rets.tail())

               SP500  Inflation-Linked Bonds  Nominal Bonds  Real Estate  \
Date                                                                       
2020-08-08  0.000000                0.000000       0.000000     0.000000   
2020-08-09  0.000000                0.000000       0.000000     0.000000   
2020-08-10  0.002742               -0.001189      -0.001119     0.001578   
2020-08-11 -0.007969               -0.002222      -0.003472    -0.014421   
2020-08-12  0.013997                0.000636      -0.002586     0.010082   

                Gold       BTC       ETH  GLP  TriCrypto  
Date                                                      
2020-08-08  0.000000  0.013151  0.038140  NaN        NaN  
2020-08-09  0.000000 -0.006662 -0.007277  NaN        NaN  
2020-08-10 -0.003459  0.017333  0.012188  NaN        NaN  
2020-08-11 -0.053694 -0.039365 -0.039161  NaN        NaN  
2020-08-12 -0.004668  0.015285  0.027972  NaN        NaN   


               SP500  Inflation-Linked Bonds  Nomin

In [17]:
print(monthly_rets.head(), '\n\n')
print(monthly_rets.tail())

               SP500  Inflation-Linked Bonds  Nominal Bonds  Real Estate  \
Date                                                                       
2020-09-30 -0.039228               -0.003780      -0.000985    -0.026643   
2020-10-31 -0.027666               -0.006765      -0.005564    -0.030015   
2020-11-30  0.107546                0.012072       0.012135     0.096749   
2020-12-31  0.037121                0.010976       0.001526     0.027340   
2021-01-31 -0.011137                0.002664      -0.008618     0.000353   

                Gold       BTC       ETH  GLP  TriCrypto  
Date                                                      
2020-09-30 -0.041714 -0.076735 -0.172708  NaN        NaN  
2020-10-31 -0.005194  0.277853  0.074047  NaN        NaN  
2020-11-30 -0.054086  0.424123  0.590424  NaN        NaN  
2020-12-31  0.070139  0.477732  0.199988  NaN        NaN  
2021-01-31 -0.032238  0.141807  0.782299  NaN        NaN   


               SP500  Inflation-Linked Bonds  Nomin

In [18]:
print(weekly_rets.head(), '\n\n')
print(weekly_rets.tail())

               SP500  Inflation-Linked Bonds  Nominal Bonds  Real Estate  \
Date                                                                       
2020-08-16  0.006436               -0.006975      -0.011524    -0.015050   
2020-08-23  0.007207                0.006066       0.004754    -0.003327   
2020-08-30  0.032630                0.002856      -0.006421     0.021143   
2020-09-06 -0.023104               -0.001108       0.002073    -0.008354   
2020-09-13 -0.025092                0.001822       0.002040    -0.020147   

                Gold       BTC       ETH  GLP  TriCrypto  
Date                                                      
2020-08-16 -0.043342  0.018591  0.109087  NaN        NaN  
2020-08-23 -0.002794 -0.019168 -0.097749  NaN        NaN  
2020-08-30  0.012965  0.004000  0.094565  NaN        NaN  
2020-09-06 -0.014914 -0.122201 -0.175150  NaN        NaN  
2020-09-13  0.004459  0.004222  0.034547  NaN        NaN   


               SP500  Inflation-Linked Bonds  Nomin

## Calculate Monthly Excess Returns

In [19]:
monthly_rets = monthly_rets.join(rfs)
for col in monthly_rets.columns.drop('RF'):
    newcol = col + ' - ' + 'RF'
    monthly_rets[newcol] = monthly_rets[col] - monthly_rets['RF']

In [20]:
monthly_rets.loc[:, monthly_rets.columns.str.endswith('- RF')]

,SP500 - RF,Inflation-Linked Bonds - RF,Nominal Bonds - RF,Real Estate - RF,Gold - RF,BTC - RF,ETH - RF,GLP - RF,TriCrypto - RF
Date,,,,,,,,,
2020-09-30,-0.049228,-0.013780,-0.010985,-0.036643,-0.051714,-0.086735,-0.182708,NaN,NaN
2020-10-31,-0.037666,-0.016765,-0.015564,-0.040015,-0.015194,0.267853,0.064047,NaN,NaN
2020-11-30,0.097546,0.002072,0.002135,0.086749,-0.064086,0.414123,0.580424,NaN,NaN
2020-12-31,0.027121,0.000976,-0.008474,0.017340,0.060139,0.467732,0.189988,NaN,NaN
2021-01-31,-0.021137,-0.007336,-0.018618,-0.009647,-0.042238,0.131807,0.772299,NaN,NaN
2021-02-28,0.026091,-0.016876,-0.015477,0.034251,-0.062569,0.363088,0.076855,NaN,NaN
2021-03-31,0.042439,-0.002622,-0.012714,0.051520,-0.011433,0.305311,0.354729,NaN,NaN
2021-04-30,0.052425,0.014297,0.008660,0.078598,0.035634,-0.019835,0.445612,NaN,NaN
2021-05-31,0.005486,0.010239,0.001539,0.008074,0.076784,-0.353546,-0.021009,NaN,NaN


In [21]:
# for a fair comparison, we want to ensure all assets have the same months. GLP and TriCrypto have the least 
# amount of history. It's misleading to compare, for example, BTC's beta calculated using more historical months with 
# GLP or TriCrypto's beta calculated using fewer months. 
excess_monthly_rets = monthly_rets.dropna().loc[:, monthly_rets.columns.str.endswith('- RF')]
excess_monthly_rets

,SP500 - RF,Inflation-Linked Bonds - RF,Nominal Bonds - RF,Real Estate - RF,Gold - RF,BTC - RF,ETH - RF,GLP - RF,TriCrypto - RF
Date,,,,,,,,,
2021-09-30,-0.047569,-0.007801,-0.010136,-0.056815,-0.032235,-0.071572,-0.125826,-0.036346,-0.067016
2021-10-31,0.069144,0.011096,0.000703,0.071330,0.014797,0.400267,0.428559,0.146687,0.261740
2021-11-30,-0.008334,0.008650,0.002041,-0.021093,-0.006901,-0.070346,0.080084,0.017250,0.006004
2021-12-31,0.033613,-0.005971,-0.013103,0.087056,0.022991,-0.197684,-0.214869,-0.075566,-0.138438
2022-01-31,-0.052585,-0.020588,-0.020649,-0.084217,-0.016788,-0.168947,-0.270012,-0.213344,-0.185582
2022-02-28,-0.031360,0.008557,-0.011380,-0.034827,0.061217,0.122394,0.085900,0.062604,0.066120
2022-03-31,0.025773,-0.028755,-0.037521,0.052639,0.002726,0.044301,0.114158,0.057190,0.086974
2022-04-30,-0.097957,-0.031831,-0.049727,-0.050878,-0.030703,-0.181806,-0.178043,-0.078290,-0.111542
2022-05-31,-0.029947,-0.039922,-0.021703,-0.076854,-0.062615,-0.187035,-0.318573,-0.128967,-0.201612


In [22]:
print('Data period: ', excess_monthly_rets.index.min().strftime('%Y-%m-%d'), 
      '~', excess_monthly_rets.index.max().strftime('%Y-%m-%d'))

Data period:  2021-09-30 ~ 2022-06-30


## Calculate Beta, Sharpe Ratio, and Excess Return (Ann) using Excess Monthly Returns

- Treat SP500 as benchmark.
- GLP and TriCrypto Yields are excluded.

In [23]:
market = 'SP500 - RF'
tokens = excess_monthly_rets.columns.drop(market)
betas = [calc_beta(excess_monthly_rets, token, market).round(3) for token in tokens]
pd.Series(betas, index=tokens).sort_values().to_frame().rename({0:'beta'}, axis=1)

,beta
Nominal Bonds - RF,0.289
Inflation-Linked Bonds - RF,0.356
Gold - RF,0.410
Real Estate - RF,1.039
GLP - RF,1.446
TriCrypto - RF,2.198
BTC - RF,2.690
ETH - RF,3.261


In [24]:
sharpe_ratios = (excess_monthly_rets.mean() / excess_monthly_rets.std()).round(3)
sharpe_ratios.sort_values(ascending=False).to_frame().rename({0:'Sharpe Ratio'}, axis=1)

,Sharpe Ratio
Gold - RF,-0.302
BTC - RF,-0.331
ETH - RF,-0.337
Real Estate - RF,-0.340
TriCrypto - RF,-0.367
GLP - RF,-0.394
SP500 - RF,-0.449
Inflation-Linked Bonds - RF,-0.639
Nominal Bonds - RF,-0.973


In [25]:
tot_ret = (1+excess_monthly_rets).prod()-1
dur_years = len(excess_monthly_rets) / 12
ann_excess_rets = annualize_tot_ret(tot_ret, dur_years).round(3)
ann_excess_rets.sort_values(ascending=False).to_frame().rename({0:'Excess Return (Ann)'}, axis=1)

,Excess Return (Ann)
Gold - RF,-0.146
Inflation-Linked Bonds - RF,-0.217
Nominal Bonds - RF,-0.254
Real Estate - RF,-0.282
SP500 - RF,-0.308
GLP - RF,-0.499
TriCrypto - RF,-0.640
BTC - RF,-0.713
ETH - RF,-0.805


## Calculate Betas, Sharpe Ratio, and Return (Ann) using Weekly Returns

- Weekly risk free rates are unavailable so we can't calculate excess weekly returns.
- Treat SP500 as benchmark.
- GLP and TriCrypto Yields are excluded.

In [26]:
market = 'SP500'
tokens = weekly_rets.columns.drop(market)
betas = [calc_beta(weekly_rets, token, market).round(3) for token in tokens]
pd.Series(betas, index=tokens).sort_values().to_frame().rename({0:'beta'}, axis=1)

,beta
Nominal Bonds,0.044
Inflation-Linked Bonds,0.057
Gold,0.136
Real Estate,0.776
GLP,1.118
TriCrypto,1.473
BTC,1.788
ETH,2.523


In [27]:
sharpe_ratios = (weekly_rets.mean() / weekly_rets.std()).round(3)
sharpe_ratios.sort_values(ascending=False).to_frame().rename({0:'Sharpe Ratio'}, axis=1)

,Sharpe Ratio
ETH,0.174
BTC,0.116
Real Estate,0.100
SP500,0.096
Inflation-Linked Bonds,0.010
TriCrypto,-0.025
Gold,-0.062
GLP,-0.103
Nominal Bonds,-0.173


In [28]:
tot_ret = (1+weekly_rets).prod()-1
dur_years = len(weekly_rets) / 52
ann_rets = annualize_tot_ret(tot_ret, dur_years).round(3)
ann_rets.sort_values(ascending=False).to_frame().rename({0:'Return (Ann)'}, axis=1)

,Return (Ann)
ETH,1.084
BTC,0.409
Real Estate,0.122
SP500,0.112
Inflation-Linked Bonds,0.002
Nominal Bonds,-0.057
Gold,-0.069
TriCrypto,-0.107
GLP,-0.144
